# 1  Problem Statement

If they are more likely to be controlled the more frequent they visit and if that changes based on patient demographics like with insurance without insurance etc
We generally tell diabetics every 3 months want to see if that makes a difference with diabetes and last office visit and control vs. uncontrolled numbers. 


1.       PatientID repeats because they have more than one visit date and more than one date when their a1c was recorded.

2.       Summary = Document or office visit summary associated with the visit. I don’t know if that would be a HIPPA violation, but there is no patient information on the summary so I am guessing it should be good.

3.       VisitDate , summary, date of a1c  has repeated values because the date when a1c was recorded and the visit date are not necessarily the same and patient has different visit dates and different date of a1c, basically its two data sets one would be patientid, summary and visit and the other would be Patientid, date of a1c and a1c_value, Combining those two will create duplicates.

1. Are patients more likely to have a lower/controlled A1c if they have visited more frequently? 
2. NLP on summary: words like poor/worse 
3. Can we predict based off of frequency of visits or last visit, what the patients A1c is? 
- people who come in more frequently, more inclined to have lower levels 
- controlled randomised experiment 

# Imports

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV,Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)
%matplotlib inline

In [5]:
df=pd.read_csv("../data/A1c_Data_Requesta1c.csv")
#with insurance

In [ ]:
df_2=pd.read_csv("../data/diabetes.csv")
#without insurance and age

In [ ]:
df.head(20)

In [ ]:
df.head(25)

# Cleaning and EDA

- Discovered 1,099 patients to look at
-Changed column names
-Changed object type to integer for a1c levels 
-Removed 1.7 a1c
-Changed .089 to 8.9 -- assume an error, 8.9 also the mean value 
-Change date object to datetime 
-drop index 

Moving forward
- group a1c into 8 and below= controlled
- 9 and above = uncontrolled
- add as another column and save into csv 
- You can ignore it and assumed that the visit date is the same as the day they get their A1c- inconsistent 
- group a few patients of different age groups and export to CVS to do EDA 


In [3]:
#Check number of patients by ID
df["PatientId"].value_counts()

115425    6072
971       5880
386       5247
111755    4422
201709    3410
          ... 
234272       2
241167       2
233853       1
234976       1
233208       1
Name: PatientId, Length: 1099, dtype: int64

In [4]:
#change column names 
df.rename(columns={"PatientId": "patient_id","Age": "age", "VisitDate": "visit_date", "SUMMARY":"summary","OBSDATE":"obs_date","Date of A1c":"date_of_a1c","Insurance":"insurance"},inplace=True)

In [5]:
df.head()

,patient_id,age,Birthdate,visit_date,summary,obs_date,date_of_a1c,insurance
0,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",7/21/2017 17:21:00,8.1,Dental Sliding Fee
1,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",7/21/2017 17:21:00,8.1,Sliding Fee Scale
2,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",1/19/2019 0:01:15,7.6,Dental Sliding Fee
3,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",1/19/2019 0:01:15,7.6,Sliding Fee Scale
4,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",2/8/2018 19:50:00,8.3,Dental Sliding Fee


In [6]:
df.shape

(273485, 8)

In [7]:
df.dtypes

patient_id       int64
age              int64
Birthdate       object
visit_date      object
summary         object
obs_date        object
date_of_a1c    float64
insurance       object
dtype: object

In [8]:
df.isnull().sum()

patient_id       0
age              0
Birthdate        0
visit_date       0
summary        218
obs_date         0
date_of_a1c     32
insurance        0
dtype: int64

In [9]:
df.dropna(subset=["date_of_a1c"],inplace=True)

In [10]:
df.reset_index(inplace=True)

In [11]:
df.isnull().sum()

index            0
patient_id       0
age              0
Birthdate        0
visit_date       0
summary        218
obs_date         0
date_of_a1c      0
insurance        0
dtype: int64

In [12]:
#convert object to float 
def convert(a1c_obj):
    a1c_num=float(a1c_obj)
    return a1c_num

In [13]:
df["date_of_a1c"].map(convert)

0          8.1
1          8.1
2          7.6
3          7.6
4          8.3
          ... 
273448    13.4
273449    13.4
273450    12.2
273451    12.2
273452    13.4
Name: date_of_a1c, Length: 273453, dtype: float64

In [14]:
df["date_of_a1c"].sort_values(ascending=True)

204598     0.089
204659     0.089
204658     0.089
204651     0.089
204650     0.089
           ...  
268928    19.700
268929    19.700
268930    19.700
268931    19.700
268927    19.700
Name: date_of_a1c, Length: 273453, dtype: float64

In [15]:
df["date_of_a1c"].value_counts(ascending=True)

19.700        5
15.200       16
17.000       20
15.400       34
17.100       39
15.300       40
16.900       44
0.089        45
13.600       60
14.300      107
14.500      115
14.900      116
14.600      120
15.000      120
13.200      122
15.500      164
14.700      180
13.800      202
13.500      223
15.600      224
13.900      234
16.700      252
12.800      264
14.100      298
14.800      317
14.200      346
14.000      354
14.400      427
13.100      437
1.700       552
11.900      573
13.700      579
13.400      627
12.900      719
12.700      733
13.300      863
12.300     1010
12.600     1044
11.600     1097
13.000     1267
11.700     1413
12.000     1456
12.400     1526
10.900     1613
12.500     1615
11.300     1772
12.200     1847
11.500     1859
11.800     1893
12.100     1942
11.000     2001
11.400     2134
11.200     2235
10.600     2357
11.100     2528
10.800     2679
9.800      2734
10.400     2749
10.300     2961
10.000     3273
10.700     3307
9.000      3422
9.700   

In [16]:
df["date_of_a1c"].max()

19.7

In [17]:
df["date_of_a1c"].min()
#this minimum is too low, assume this was meant to be 8.9, which is the mean of the dataset 

0.08900000000000001

In [18]:
df["date_of_a1c"].describe()

count    273453.000000
mean          8.902426
std           1.760240
min           0.089000
25%           7.500000
50%           8.400000
75%           9.900000
max          19.700000
Name: date_of_a1c, dtype: float64

In [19]:
mask_1= df["date_of_a1c"]== 0.08900000000000001
#instead of deleting, change 

In [20]:
mask_1

0         False
1         False
2         False
3         False
4         False
          ...  
273448    False
273449    False
273450    False
273451    False
273452    False
Name: date_of_a1c, Length: 273453, dtype: bool

In [21]:
df["date_of_a1c"].where(df["date_of_a1c"]!= 0.08900000000000001, 8.9,inplace=True)

In [22]:
df.head()

,index,patient_id,age,Birthdate,visit_date,summary,obs_date,date_of_a1c,insurance
0,0,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",7/21/2017 17:21:00,8.1,Dental Sliding Fee
1,1,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",7/21/2017 17:21:00,8.1,Sliding Fee Scale
2,2,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",1/19/2019 0:01:15,7.6,Dental Sliding Fee
3,3,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",1/19/2019 0:01:15,7.6,Sliding Fee Scale
4,4,72,56,11/26/1963 5:00:00,7/19/2017 14:30:00,"followup diabetes, med refill",2/8/2018 19:50:00,8.3,Dental Sliding Fee


In [23]:
df["date_of_a1c"].value_counts(ascending=True)

19.7        5
15.2       16
17.0       20
15.4       34
17.1       39
15.3       40
16.9       44
13.6       60
14.3      107
14.5      115
14.9      116
15.0      120
14.6      120
13.2      122
15.5      164
14.7      180
13.8      202
13.5      223
15.6      224
13.9      234
16.7      252
12.8      264
14.1      298
14.8      317
14.2      346
14.0      354
14.4      427
13.1      437
1.7       552
11.9      573
13.7      579
13.4      627
12.9      719
12.7      733
13.3      863
12.3     1010
12.6     1044
11.6     1097
13.0     1267
11.7     1413
12.0     1456
12.4     1526
10.9     1613
12.5     1615
11.3     1772
12.2     1847
11.5     1859
11.8     1893
12.1     1942
11.0     2001
11.4     2134
11.2     2235
10.6     2357
11.1     2528
10.8     2679
9.8      2734
10.4     2749
10.3     2961
10.0     3273
10.7     3307
9.0      3422
9.7      3426
9.9      3504
10.2     3660
10.1     3728
8.9      3943
9.3      4019
10.5     4145
9.2      4305
9.5      4344
9.4      4443
9.1   

In [24]:
df.shape

(273453, 9)

In [25]:
mask_2= df["date_of_a1c"]== 1.7

In [26]:
mask_2.value_counts()

False    272901
True        552
Name: date_of_a1c, dtype: int64

In [27]:
#delete values that don't make sense in df
def to_del(value_to_delete):
    to_del_array=[]
    for i in range(len(value_to_delete)):
        if value_to_delete[i]== True: 
            to_del_array.append(i)
    df.drop(to_del_array,inplace=True)
    df.reset_index(drop=True,inplace=True)

In [28]:
to_del(mask_2)

In [29]:
df.shape

(272901, 9)

In [30]:
df["date_of_a1c"].min()

7.0

In [31]:
df["date_of_a1c"].value_counts(ascending=True)

19.7        5
15.2       16
17.0       20
15.4       34
17.1       39
15.3       40
16.9       44
13.6       60
14.3      107
14.5      115
14.9      116
14.6      120
15.0      120
13.2      122
15.5      164
14.7      180
13.8      202
13.5      223
15.6      224
13.9      234
16.7      252
12.8      264
14.1      298
14.8      317
14.2      346
14.0      354
14.4      427
13.1      437
11.9      573
13.7      579
13.4      627
12.9      719
12.7      733
13.3      863
12.3     1010
12.6     1044
11.6     1097
13.0     1267
11.7     1413
12.0     1456
12.4     1526
10.9     1613
12.5     1615
11.3     1772
12.2     1847
11.5     1859
11.8     1893
12.1     1942
11.0     2001
11.4     2134
11.2     2235
10.6     2357
11.1     2528
10.8     2679
9.8      2734
10.4     2749
10.3     2961
10.0     3273
10.7     3307
9.0      3422
9.7      3426
9.9      3504
10.2     3660
10.1     3728
8.9      3943
9.3      4019
10.5     4145
9.2      4305
9.5      4344
9.4      4443
9.1      4475
9.6   

In [32]:
df["insurance"].value_counts()

Sliding Fee Scale                                62917
Medicaid - Husky D                               28659
Medicaid - Husky C                               24322
DO NOT USE - Medicaid State of CT                23376
National Government Services                     21913
Dental Sliding Fee                               17748
Medicaid - Husky A                               15554
Connecticare Exchange                             6944
DO NOT USE-Community Health Network - Husky A     6363
Medicaid State of CT                              6318
DO NOT USE - CHN Saga- Medicaid LIA               6174
Anthem Medicare                                   5742
United Healthcare Medicare                        4564
Connecticare                                      4546
CT Behavioral Health Partnership                  4216
Anthem BCBS                                       4010
Aetna Commercial                                  2675
Aetna Medicare -HMO                               2657
CT Dental 

In [33]:
df["visit_date"].head()

0    7/19/2017 14:30:00
1    7/19/2017 14:30:00
2    7/19/2017 14:30:00
3    7/19/2017 14:30:00
4    7/19/2017 14:30:00
Name: visit_date, dtype: object

In [34]:
df['visit_date'] = pd.to_datetime(df['visit_date'])

In [35]:
df['obs_date'] = pd.to_datetime(df['obs_date'])

In [36]:
df.head()

,index,patient_id,age,Birthdate,visit_date,summary,obs_date,date_of_a1c,insurance
0,0,72,56,11/26/1963 5:00:00,2017-07-19 14:30:00,"followup diabetes, med refill",2017-07-21 17:21:00,8.1,Dental Sliding Fee
1,1,72,56,11/26/1963 5:00:00,2017-07-19 14:30:00,"followup diabetes, med refill",2017-07-21 17:21:00,8.1,Sliding Fee Scale
2,2,72,56,11/26/1963 5:00:00,2017-07-19 14:30:00,"followup diabetes, med refill",2019-01-19 00:01:15,7.6,Dental Sliding Fee
3,3,72,56,11/26/1963 5:00:00,2017-07-19 14:30:00,"followup diabetes, med refill",2019-01-19 00:01:15,7.6,Sliding Fee Scale
4,4,72,56,11/26/1963 5:00:00,2017-07-19 14:30:00,"followup diabetes, med refill",2018-02-08 19:50:00,8.3,Dental Sliding Fee


In [37]:
df.dtypes

index                   int64
patient_id              int64
age                     int64
Birthdate              object
visit_date     datetime64[ns]
summary                object
obs_date       datetime64[ns]
date_of_a1c           float64
insurance              object
dtype: object

In [38]:
df["age"].value_counts()

59    20338
65    14184
64    13715
61    11974
69     9924
57     9588
73     8664
60     8546
62     7687
66     7466
67     7381
53     7284
68     7240
70     7198
50     6995
63     6883
54     6401
58     6380
49     6292
75     6180
74     5998
81     5774
56     5529
51     5397
55     4990
45     4950
47     4629
46     4384
52     4225
71     3676
87     3404
72     3306
79     2944
77     2675
41     2593
48     2389
80     1892
84     1808
44     1681
37     1515
34     1474
39     1463
78     1240
76     1222
38     1203
32     1193
90     1104
35      956
88      839
83      820
92      800
40      754
42      729
33      619
27      552
31      522
20      496
36      363
43      328
23      318
25      308
24      240
29      210
89      200
96      180
82      147
26       99
28       94
93       90
95       68
86       68
85       45
18       36
21       20
30       10
19        6
3         4
4         2
Name: age, dtype: int64

In [39]:
df.loc[df["age"]==3]

,index,patient_id,age,Birthdate,visit_date,summary,obs_date,date_of_a1c,insurance
252373,252957,230569,3,3/16/2016 4:00:00,2018-06-27 10:44:00,NaN,2017-08-28 15:33:46,7.7,CT Behavioral Health Partnership
252374,252958,230569,3,3/16/2016 4:00:00,2018-06-27 10:44:00,NaN,2017-08-28 15:33:46,7.7,Sliding Fee Scale
252375,252959,230569,3,3/16/2016 4:00:00,2018-06-21 14:18:00,NaN,2017-08-28 15:33:46,7.7,CT Behavioral Health Partnership
252376,252960,230569,3,3/16/2016 4:00:00,2018-06-21 14:18:00,NaN,2017-08-28 15:33:46,7.7,Sliding Fee Scale


In [40]:
#take patient_id and convert to dataframe/csv 
df.loc[df["patient_id"]==230569]

,index,patient_id,age,Birthdate,visit_date,summary,obs_date,date_of_a1c,insurance
252373,252957,230569,3,3/16/2016 4:00:00,2018-06-27 10:44:00,NaN,2017-08-28 15:33:46,7.7,CT Behavioral Health Partnership
252374,252958,230569,3,3/16/2016 4:00:00,2018-06-27 10:44:00,NaN,2017-08-28 15:33:46,7.7,Sliding Fee Scale
252375,252959,230569,3,3/16/2016 4:00:00,2018-06-21 14:18:00,NaN,2017-08-28 15:33:46,7.7,CT Behavioral Health Partnership
252376,252960,230569,3,3/16/2016 4:00:00,2018-06-21 14:18:00,NaN,2017-08-28 15:33:46,7.7,Sliding Fee Scale


In [41]:
df.loc[df["age"]==18]

,index,patient_id,age,Birthdate,visit_date,summary,obs_date,date_of_a1c,insurance
43745,43745,111445,18,4/16/2001 4:00:00,2019-11-14 13:45:00,"DM(type?)2 f/u, ref Nutrition, pt refuses med ...",2019-12-10 13:04:27,9.3,DO NOT USE-Community Health Network - Husky A
43746,43746,111445,18,4/16/2001 4:00:00,2019-11-14 13:45:00,"DM(type?)2 f/u, ref Nutrition, pt refuses med ...",2019-12-10 13:04:27,9.3,Medicaid - Husky A
43747,43747,111445,18,4/16/2001 4:00:00,2019-11-14 13:45:00,"DM(type?)2 f/u, ref Nutrition, pt refuses med ...",2019-12-10 13:04:27,9.3,United Healthcare Commercial
43748,43748,111445,18,4/16/2001 4:00:00,2019-11-14 13:45:00,"DM(type?)2 f/u, ref Nutrition, pt refuses med ...",2019-12-10 13:04:27,9.3,DO NOT USE Blue Care Family Plan
43749,43749,111445,18,4/16/2001 4:00:00,2019-11-14 13:45:00,"DM(type?)2 f/u, ref Nutrition, pt refuses med ...",2019-12-10 13:04:27,9.3,Dental Sliding Fee
43750,43750,111445,18,4/16/2001 4:00:00,2019-11-14 13:45:00,"DM(type?)2 f/u, ref Nutrition, pt refuses med ...",2019-12-10 13:04:27,9.3,Sliding Fee Scale
43751,43751,111445,18,4/16/2001 4:00:00,2019-12-10 15:15:00,"DM2 f/u (A1c 9.2), add trulicity, periorbital ...",2019-12-10 13:04:27,9.3,DO NOT USE-Community Health Network - Husky A
43752,43752,111445,18,4/16/2001 4:00:00,2019-12-10 15:15:00,"DM2 f/u (A1c 9.2), add trulicity, periorbital ...",2019-12-10 13:04:27,9.3,Medicaid - Husky A
43753,43753,111445,18,4/16/2001 4:00:00,2019-12-10 15:15:00,"DM2 f/u (A1c 9.2), add trulicity, periorbital ...",2019-12-10 13:04:27,9.3,United Healthcare Commercial
43754,43754,111445,18,4/16/2001 4:00:00,2019-12-10 15:15:00,"DM2 f/u (A1c 9.2), add trulicity, periorbital ...",2019-12-10 13:04:27,9.3,DO NOT USE Blue Care Family Plan


In [ ]:
df.loc[df["age"]==32]

In [ ]:
# #ad a column controlled and ucontrolled 
# df["ac1_level"]= [for i in range(df.shape[0]) "controlled" if df["date_of_a1c"]<=8 else "uncontrolled" ]
df["ac1_level"]=""

In [42]:
df.shape

(272901, 9)

In [43]:
df.drop_duplicates(inplace=True)

In [46]:
df_no_ins= df.drop(columns=["insurance"])

In [58]:
df_no_ins.drop(columns=["Birthdate"],inplace=True)

In [52]:
df_no_ins['obs_date'] = df_no_ins['obs_date'].dt.date
df_no_ins['visit_date'] = df_no_ins['visit_date'].dt.date

In [59]:
df_no_ins.drop_duplicates(inplace=True)

In [61]:
df_no_ins.drop(columns=["index"],inplace=True)

In [63]:
df_no_ins.drop_duplicates(inplace=True)

In [73]:
df_no_ins.reset_index(drop=True,inplace=True)

In [74]:
df_no_ins["date_of_a1c"][1]

7.6

In [75]:
#create a column that describes patient as being controlled or uncontrolled 

df_no_ins["ac1_level"]=""

for i in range(df_no_ins.shape[0]):
    if df_no_ins["date_of_a1c"][i] <=8:
        df_no_ins["ac1_level"][i]="controlled"
    else: df_no_ins["ac1_level"][i]="uncontrolled"

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [77]:
df_no_ins.head()

,patient_id,age,visit_date,summary,obs_date,date_of_a1c,ac1_level
0,72,56,2017-07-19,"followup diabetes, med refill",2017-07-21,8.1,uncontrolled
1,72,56,2017-07-19,"followup diabetes, med refill",2019-01-19,7.6,controlled
2,72,56,2017-07-19,"followup diabetes, med refill",2018-02-08,8.3,uncontrolled
3,72,56,2017-07-19,"followup diabetes, med refill",2017-10-20,7.7,controlled
4,72,56,2017-07-19,"followup diabetes, med refill",2019-06-08,7.1,controlled


In [78]:
np.unique(df_no_ins["patient_id"].values)

array([    72,    254,    255, ..., 243548, 243642, 243644])

In [79]:
df_no_ins.loc[df_no_ins["patient_id"]==72].sort_values(by=["visit_date","obs_date"])

,patient_id,age,visit_date,summary,obs_date,date_of_a1c,ac1_level
0,72,56,2017-07-19,"followup diabetes, med refill",2017-07-21,8.1,uncontrolled
3,72,56,2017-07-19,"followup diabetes, med refill",2017-10-20,7.7,controlled
2,72,56,2017-07-19,"followup diabetes, med refill",2018-02-08,8.3,uncontrolled
1,72,56,2017-07-19,"followup diabetes, med refill",2019-01-19,7.6,controlled
4,72,56,2017-07-19,"followup diabetes, med refill",2019-06-08,7.1,controlled
5,72,56,2017-07-19,"followup diabetes, med refill",2019-12-09,8.2,uncontrolled
11,72,56,2017-07-28,"f/u lab results, DM2, HLD",2017-07-21,8.1,uncontrolled
9,72,56,2017-07-28,"f/u lab results, DM2, HLD",2017-10-20,7.7,controlled
8,72,56,2017-07-28,"f/u lab results, DM2, HLD",2018-02-08,8.3,uncontrolled
10,72,56,2017-07-28,"f/u lab results, DM2, HLD",2019-01-19,7.6,controlled


In [80]:
df_no_ins.dtypes

patient_id       int64
age              int64
visit_date      object
summary         object
obs_date        object
date_of_a1c    float64
ac1_level       object
dtype: object

In [81]:
df_no_ins.to_csv("../data/cleaned_data.csv")

In [6]:
new_data= pd.read_csv("../data/cleaned_data.csv")

In [8]:
#loop through visits and group number of visits 
new_data.drop(columns=["Unnamed: 0"],inplace=True)

VisitDate , summary, date of a1c has repeated values because the date when a1c was recorded and the visit date are not necessarily the same and patient has different visit dates and different date of a1c, basically its two data sets one would be patientid, summary and visit and the other would be Patientid, date of a1c and a1c_value, Combining those two will create duplicates.

In [9]:
new_data.dtypes

patient_id       int64
age              int64
visit_date      object
summary         object
obs_date        object
date_of_a1c    float64
ac1_level       object
dtype: object

In [86]:
new_data.loc[new_data["patient_id"]==72].sort_values(by=["visit_date","obs_date"])

,patient_id,age,visit_date,summary,obs_date,date_of_a1c,ac1_level
0,72,56,2017-07-19,"followup diabetes, med refill",2017-07-21,8.1,uncontrolled
3,72,56,2017-07-19,"followup diabetes, med refill",2017-10-20,7.7,controlled
2,72,56,2017-07-19,"followup diabetes, med refill",2018-02-08,8.3,uncontrolled
1,72,56,2017-07-19,"followup diabetes, med refill",2019-01-19,7.6,controlled
4,72,56,2017-07-19,"followup diabetes, med refill",2019-06-08,7.1,controlled
5,72,56,2017-07-19,"followup diabetes, med refill",2019-12-09,8.2,uncontrolled
11,72,56,2017-07-28,"f/u lab results, DM2, HLD",2017-07-21,8.1,uncontrolled
9,72,56,2017-07-28,"f/u lab results, DM2, HLD",2017-10-20,7.7,controlled
8,72,56,2017-07-28,"f/u lab results, DM2, HLD",2018-02-08,8.3,uncontrolled
10,72,56,2017-07-28,"f/u lab results, DM2, HLD",2019-01-19,7.6,controlled


In [10]:
new_data['visit_date'] = pd.to_datetime(new_data['visit_date'])

In [11]:
new_data['obs_date'] = pd.to_datetime(new_data['obs_date'])

In [15]:
new_data.head()

,patient_id,age,visit_date,summary,obs_date,date_of_a1c,ac1_level,num_obs
0,72,56,2017-07-19,"followup diabetes, med refill",2017-07-21,8.1,uncontrolled,6
1,72,56,2017-07-19,"followup diabetes, med refill",2019-01-19,7.6,controlled,6
2,72,56,2017-07-19,"followup diabetes, med refill",2018-02-08,8.3,uncontrolled,6
3,72,56,2017-07-19,"followup diabetes, med refill",2017-10-20,7.7,controlled,6
4,72,56,2017-07-19,"followup diabetes, med refill",2019-06-08,7.1,controlled,6


In [14]:
new_data["num_obs"]=""

for i in range(new_data.shape[0]):
    try:
        mas=new_data.loc[new_data['patient_id'] == new_data['patient_id'][i]]
        new_data["num_obs"][i]= mas['obs_date'].nunique()
    except: 
        continue 

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
new_data["num_visits"]=""

for i in range(new_data.shape[0]):
    try:
        mas=new_data.loc[new_data['patient_id'] == new_data['patient_id'][i]]
        new_data["num_visits"][i]= mas["visit_date"].nunique()
    except: 
        continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
summary_and_visit_df = new_data[['patient_id', 'age', 'visit_date','summary',"num_visits"]].copy()
date_a1c_df= new_data[['patient_id', 'age', 'obs_date','date_of_a1c','ac1_level',"num_obs"]].copy()

In [21]:
summary_and_visit_df.head()

,patient_id,age,visit_date,summary,num_visits
0,72,56,2017-07-19,"followup diabetes, med refill",10
1,72,56,2017-07-19,"followup diabetes, med refill",10
2,72,56,2017-07-19,"followup diabetes, med refill",10
3,72,56,2017-07-19,"followup diabetes, med refill",10
4,72,56,2017-07-19,"followup diabetes, med refill",10


In [23]:
summary_and_visit_df.shape[0] == date_a1c_df.shape[0]

True

In [24]:
summary_and_visit_df.shape

(61943, 5)

In [25]:
date_a1c_df.shape

(61943, 6)

In [26]:
summary_and_visit_df.drop_duplicates(inplace=True)

In [27]:
summary_and_visit_df.shape

(14197, 5)

In [37]:
summary_and_visit_df.to_csv("../data/sum_visit")

In [38]:
date_a1c_df.to_csv("../data/sum_obs")

In [33]:
summary_and_visit_df.loc[summary_and_visit_df["patient_id"]==72]

,patient_id,age,visit_date,summary,num_visits
0,72,56,2017-07-19,"followup diabetes, med refill",10
6,72,56,2017-07-28,"f/u lab results, DM2, HLD",10
12,72,56,2017-11-08,DM and dyslipidemia follow-up,10
18,72,56,2018-02-16,Uncontrolled DM,10
24,72,56,2018-05-02,DM - hypoglycemia- dec. glyburide,10
30,72,56,2018-09-28,"Poorly controlled DM, osteoarthritis right thu...",10
36,72,56,2019-01-18,Diabetes follow up,10
40,72,56,2019-05-17,"f/u DM2, HLD",10
44,72,56,2019-12-09,"DM (A1c 7.1%), needs PPSV23/PCV13 vaccines",10
48,72,56,2020-01-03,"Worse a1c 8.2 from 7, no new meds, repeat lipids",10


In [35]:
date_a1c_df.loc[date_a1c_df["patient_id"]==72]

,patient_id,age,obs_date,date_of_a1c,ac1_level,num_obs
0,72,56,2017-07-21,8.1,uncontrolled,6
1,72,56,2019-01-19,7.6,controlled,6
2,72,56,2018-02-08,8.3,uncontrolled,6
3,72,56,2017-10-20,7.7,controlled,6
4,72,56,2019-06-08,7.1,controlled,6
5,72,56,2019-12-09,8.2,uncontrolled,6


In [28]:
date_a1c_df.drop_duplicates(inplace=True)
#different insurances 

In [29]:
date_a1c_df.shape
#they visit way more than they get their bloodwork done 

(3617, 6)

In [ ]:
#if patiend id is the same and obs date no more than a week later, then associat ethose two visits together/ add column 
#frequency of visits - correlation between frequency and a1c 
#get rid of time 
# count frequnecy of visits/year - EDA with a1c levels that year since 2017 
#limitations and future steps 

In [ ]:
date_a1c_df['obs_date'] = pd.to_datetime(date_a1c_df['obs_date'])

In [ ]:
summary_and_visit_df['visit_date'] = pd.to_datetime(summary_and_visit_df['visit_date'])

In [ ]:
summary_and_visit_df["a1c_level"]=""
summary_and_visit_df["a1c_type"]=""
summary_and_visit_df["obs_date"]=""
    
for i in range(summary_and_visit_df.shape[0]):
    for j in range(date_a1c_df.shape[0]):
        try: 
            if summary_and_visit_df["patient_id"][i]==date_a1c_df["patient_id"][j]: 
                print()
                sum_dates= date_a1c_df["obs_date"][j] - summary_and_visit_df["visit_date"][i]
                if sum_dates <= np.timedelta64(2,'W'): 
                    summary_and_visit_df["a1c_level"][i] = date_a1c_df["date_of_a1c"][j]
                    summary_and_visit_df["a1c_type"][i]= date_a1c_df["ac1_level"][j]
                    summary_and_visit_df["obs_date"][i]= date_a1c_df["obs_date"][j]
        except:
            continue

In [ ]:
summary_and_visit_df.head()

In [61]:
summary_and_visit_df.dtypes

patient_id             int64
age                    int64
visit_date    datetime64[ns]
summary               object
num_visits            object
dtype: object

In [40]:
array_of_ids= np.unique(summary_and_visit_df["patient_id"].values)

In [41]:
len(np.unique(summary_and_visit_df["patient_id"].values))

1099

In [53]:
patient3= date_a1c_df.loc[date_a1c_df["patient_id"]==72]
patient_sum4=summary_and_visit_df.loc[summary_and_visit_df["patient_id"]==72]

In [63]:
patient3["obs_date"] - patient_sum4["visit_date"] <= np.timedelta64(30,'D')

0      True
1     False
2     False
3     False
4     False
5     False
6     False
12    False
18    False
24    False
30    False
36    False
40    False
44    False
48    False
dtype: bool

In [74]:
patient3["obs_date"]

0   2017-07-21
1   2019-01-19
2   2018-02-08
3   2017-10-20
4   2019-06-08
5   2019-12-09
Name: obs_date, dtype: datetime64[ns]

In [73]:
patient_sum4["visit_date"]

0    2017-07-19
6    2017-07-28
12   2017-11-08
18   2018-02-16
24   2018-05-02
30   2018-09-28
36   2019-01-18
40   2019-05-17
44   2019-12-09
48   2020-01-03
Name: visit_date, dtype: datetime64[ns]

In [72]:
patient_sum4["visit_date"][12] - patient3["obs_date"][3] <= np.timedelta64(30,'D')

True

In [69]:
patient_sum4["visit_date"][0]

Timestamp('2017-07-19 00:00:00')

In [68]:
patient_sum4["visit_date"][12]

Timestamp('2017-11-08 00:00:00')

In [70]:
patient3["obs_date"][12]

KeyError: 12

In [67]:
patient_sum4["visit_date"][12] - patient3["obs_date"][12] <= np.timedelta64(30,'D')

KeyError: 12

In [52]:
#group by patient ID and sort by dates np.unique(summary_and_visit_df["patient_id"].values)
for i in array_of_ids:
    patient= date_a1c_df.loc[date_a1c_df["patient_id"]==i]
    patient_sum=summary_and_visit_df.loc[summary_and_visit_df["patient_id"]==i]
    for row in patient: 
            if patient["obs_date"] - patient_sum["visit_date"] <= np.timedelta64(1,'M') or patient_sum["visit_date"] - patient["obs_date"] <= np.timedelta64(1,'M'):
                patient_sum["a1c_level"]=patient["date_of_a1c"]
                patient_sum["a1c_type"]=patient["ac1_level"]
                patient_sum["obs_date"]=patient["obs_date"]

In [75]:
test= date_a1c_df.loc[date_a1c_df["patient_id"]==72]
test2=summary_and_visit_df.loc[summary_and_visit_df["patient_id"]==72]

In [78]:
test2["visit_date"]

0    2017-07-19
6    2017-07-28
12   2017-11-08
18   2018-02-16
24   2018-05-02
30   2018-09-28
36   2019-01-18
40   2019-05-17
44   2019-12-09
48   2020-01-03
Name: visit_date, dtype: datetime64[ns]

In [77]:
len(test)

6

In [84]:
for dates in test2["visit_date"]:
    for dates1 in test["obs_date"]:
        if dates - dates1 <= np.timedelta64(30,'D') or dates1 - dates <= np.timedelta64(30,'D'):
            print(test["date_of_a1c"])
    

0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    7.7
4    7.1
5    8.2
Name: date_of_a1c, dtype: float64
0    8.1
1    7.6
2    8.3
3    

In [90]:
summary_and_visit_df["a1c_level"]=""
summary_and_visit_df["a1c_type"]=""
summary_and_visit_df["obs_date"]=""
summary_and_visit_df["obs_num"]=""

In [135]:
df = pd.DataFrame() 

In [158]:
for i in array_of_ids[0:2]:
    print(i)

72
254


In [149]:
date_a1c_df.head()

,patient_id,age,obs_date,date_of_a1c,ac1_level,num_obs
0,72,56,2017-07-21,8.1,uncontrolled,6
1,72,56,2019-01-19,7.6,controlled,6
2,72,56,2018-02-08,8.3,uncontrolled,6
3,72,56,2017-10-20,7.7,controlled,6
4,72,56,2019-06-08,7.1,controlled,6


In [194]:
for i in array_of_ids[0:2]:
    patient= date_a1c_df.loc[date_a1c_df["patient_id"]==i]
    patient_sum=summary_and_visit_df.loc[summary_and_visit_df["patient_id"]==i]
    for dates in patient_sum["visit_date"]:
        for dates1 in patient["obs_date"]:
            if abs(dates - dates1) <= np.timedelta64(30,'D'):
                #pick first one within those thirty days 
                temp=pd.DataFrame({ "patient_id": patient["patient_id"], "age": patient["age"],"visit_date": patient_sum["visit_date"],"summary":patient_sum["summary"],"num_visits":patient_sum["num_visits"],"a1c": patient["date_of_a1c"],"ac1_type":patient["ac1_level"],"obs_date":patient["obs_date"],"obs_num":patient["num_obs"] })
                df=pd.concat([df,temp])
            else :
                temp=pd.DataFrame({ "patient_id": patient_sum["patient_id"], "age": patient_sum["age"],"visit_date": patient_sum["visit_date"],"summary":patient_sum["summary"],"num_visits":patient_sum["num_visits"]})                
                df=pd.concat([df,temp])

-2 days +00:00:00
2017-07-19 00:00:00 2017-07-21 00:00:00


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


-1 days +00:00:00
2019-01-18 00:00:00 2019-01-19 00:00:00
0 days 00:00:00
2019-12-09 00:00:00 2019-12-09 00:00:00
0 days 00:00:00
2017-09-25 00:00:00 2017-09-25 00:00:00
-4 days +00:00:00
2018-07-23 00:00:00 2018-07-27 00:00:00
3 days 00:00:00
2018-07-30 00:00:00 2018-07-27 00:00:00
-1 days +00:00:00
2018-10-08 00:00:00 2018-10-09 00:00:00
-1 days +00:00:00
2018-12-28 00:00:00 2018-12-29 00:00:00
-4 days +00:00:00
2019-04-26 00:00:00 2019-04-30 00:00:00
0 days 00:00:00
2020-02-11 00:00:00 2020-02-11 00:00:00
